# Load the data

In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_BK as pbk; reload(pbk)
%matplotlib inline

folder_lt3 = r'D:\measuring\data\Purification_lt3_raw_data\BK_XX'
folder_lt4 = r'D:\measuring\data\Purification_lt4_raw_data\BK_XX'

### ssro calibrations
day = '20160630_' # for ssro calibration

ssro_calib_lt4 = day+'125528'
ssro_calib_lt3 = day+'142843'

BK_analysis = pbk.BK_analysis('XX',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
# a = ppq.purify_pq(folder, hdf5_mode='r')#, pq_folder='bs_remote')
# b = ppq.purify_pq(lt3_folder, hdf5_mode='r')#, pq_folder='bs_remote')

In [ ]:
### list of good timestamps and RO calibration
all_lt3,all_lt4 = [],[]

### get all valid time stamps. remember that data has to be stored locally.
msmt_days  = ['20160630','20160701','20160702']
for d in msmt_days:

    tstamp_lt3,tstamp_lt4 = BK_analysis.tstamps_for_both_setups(d)#,newest_tstamp = '110000') ### newest timestamp allows for only taking parts of a day.
    all_lt3.extend(tstamp_lt3)
    all_lt4.extend(tstamp_lt4)

    
print all_lt3
print all_lt4

In [ ]:
BK_analysis.load_raw_data(all_lt3,all_lt4)
# BK_analysis.filter_CR_after(threshold =None)

In [ ]:
BK_analysis.check_tail_w1_w2(st_start = 2498e3,st_len = 50e3,p_sep = 500e3)

In [ ]:
### autocorrelation of photon clicks --> do we have huge waiting times in the experiment
reload(pq_plots)
a_lt4 = ppq.purifyPQAnalysis(tb.latest_data(tstamp_lt4[0],folder = folder_lt4))
print a_lt4
pq_plots.plot_autocorrelation_histogram(a_lt4.pqf,start = 0,length = 200e3,binsize=0.1e3)

### Filtering and correlations

In [ ]:
photon_channel = 2 # 0 or 1; 2 means both HH detectors
st_start = 2499e3#2763e3
st_len       = 8e3 #50 ns
st_len_w2    = 8e3
#p_sep       = 2500e3 #600 ns
p_sep        =500e3#2732e3 #XXX why has this changed?
dt_max       = 8e3
ch1_offset = 0.0e3

In [ ]:
### this window can be used to analyze one of the full timetrace in order to get a feeling for the tail and the windows.
### later on all timestamps should be combined to get a clear pikkcture

a_lt4 = ppq.purifyPQAnalysis(tb.latest_data(tstamp_lt4[1],folder = folder_lt4))

vert0 = st_start/1000. ### start of the first window
vert1 = (st_start+p_sep)/1000. ### for the second window

# BK_analysis.plot_total_histogram(1,st_start-20e3,200e3,1e3,log_plot=True)
pq_plots.plot_marker_filter_comparison(a_lt4.pqf,mrkr_chan = 1,start =2470e3,  length = 50e3, hist_binsize = 0.1e3, save = False, log=False,plot_threshold_ch0 =vert0,plot_threshold_ch1 =vert1) #start =7480
# pq_plots.plot_marker_filter_comparison(a_lt4.pqf,mrkr_chan = 1,start =1965e3+1000e3,  length = 10e3, hist_binsize = 0.1e3, save = False, log=False,plot_threshold_ch0 =vert1,plot_threshold_ch1 =vert1) #start =7480
# pq_plots.plot_photon_hist(b.pqf,start =1965+500,  length = 250, hist_binsize = 1, save = False, log=True) #start =7480

In [ ]:
### return the length of events without temporal filter:

no_of_events = 0
for i in range(len(BK_analysis.lt3_dict['ssro_results'])):
    no_of_events += len(BK_analysis.lt3_dict['ssro_results'][i])

print 'number of unfiltered (temporal and CR) events', no_of_events



### apply temporal filter (see above for filters)
BK_analysis.filter_CR_after(threshold = 1,verbose = False)
BK_analysis.apply_temporal_filters_to_prefiltered_data(st_start = st_start,
                                                       st_len = st_len,
                                                       st_len_w2 = st_len_w2,
                                                       p_sep = p_sep,
                                                       dt_max = dt_max,
                                                       ch1_offset = ch1_offset)


In [ ]:
### filter the PQ data: Return an array which is True at each position where an event was in a window
BK_analysis.attach_state_filtered_syncs(verbose = False)

In [ ]:
### applies the time fitlered sync numbers to the adwin RO results of each setup
BK_analysis.correlate_RO_results(verbose = True,apply_ROC = False)

In [ ]:
### correlations vs. window start?
BK_analysis.sweep_filter_parameter_vs_correlations('st_start',np.linspace(2490e3,2510e3,20))

In [ ]:
BK_analysis.sweep_filter_parameter_vs_correlations('st_len',np.linspace(2e3,15e3,50))

In [ ]:
BK_analysis.sweep_filter_parameter_vs_correlations('ch1_offset',np.linspace(-5e3,5e3,25))

In [ ]:
BK_analysis.sweep_filter_parameter_vs_correlations('dt_max',np.linspace(2e3,5e3,15))

In [ ]:
BK_analysis.sweep_filter_parameter_vs_correlations('CR_after_threshold',np.linspace(1,20,7))

In [ ]:
#### put readout corrected analysis here.

In [ ]:
np.array([1,2,1,2])[np.array([True,False,True,False])]